In [61]:
import numpy as np
import os
import cv2
from insightface.app import FaceAnalysis
import pandas as pd
from glob import glob
from insightface.app.common import Face
from insightface.model_zoo import model_zoo
from insightface.model_zoo import model_zoo
from tqdm import tqdm
from collections import defaultdict



In [31]:
det_model_path = 'buffalo_s/det_500m.onnx'
rec_model_path = 'buffalo_s/w600k_mbf.onnx'

det_model = model_zoo.get_model(f'/Users/tayamayuto/Face_recognition_IF/.insightface/models/{det_model_path}')
rec_model = model_zoo.get_model(f'/Users/tayamayuto/Face_recognition_IF/.insightface/models/{rec_model_path}')

det_model.prepare(ctx_id=0, input_size=(640, 640), det_thres=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


/Users/tayamayuto/Face_recognition_IF/face_env/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:65: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


In [56]:
known_names = []
unknown_names = []
known_embeddings = []
unknown_embeddings = []
player_embeddings = []
player_names = []
player = 'De\'Andre Hunter'
img_paths = glob(f'/Users/tayamayuto/Face_recognition_IF/data/ts_raw/{player}/*')

for img_path in img_paths:
        img = cv2.imread(img_path)
        if img is None: continue

        bboxes, kpss = det_model.detect(img, max_num=0, metric='defualt')
        if len(bboxes) != 1: continue

        bbox = bboxes[0, :4]
        det_score = bboxes[0, 4]
        kps = kpss[0]
        face = Face(bbox=bbox, kps=kps, det_score=det_score)

        rec_model.get(img, face)
        player_embeddings.append(face.normed_embedding)
        player_names.append(player)
        if len(player_embeddings) == 10: break

print(player_embeddings)

player_embeddings = np.stack(player_embeddings, axis=0)
print(player_embeddings.shape)
known_embeddings.append(player_embeddings[0:5])
unknown_embeddings.append(player_embeddings[5:10])
known_names += player_names[0:5]
unknown_names += player_names[5:10]

[array([ 3.09586320e-02, -5.36540858e-02, -9.91005152e-02,  1.65611580e-02,
       -3.85071933e-02, -2.54213307e-02,  1.01639843e-02, -5.83655015e-03,
       -3.00230645e-03,  4.91437037e-03,  3.91675048e-02,  4.65848148e-02,
        3.81524563e-02, -6.95842644e-03,  3.11234277e-02,  7.76888877e-02,
        5.18732741e-02,  2.71901190e-02,  1.37433037e-02,  3.01586837e-02,
        1.06680961e-02, -2.63836235e-02, -2.48442218e-02,  4.32003438e-02,
        6.67094020e-03,  4.29611877e-02, -2.73890211e-03, -1.37783393e-01,
        3.62067739e-03,  9.20062065e-02, -1.35343662e-02, -4.01787758e-02,
       -2.90303845e-02,  4.90518771e-02,  3.28427516e-02,  2.32880767e-02,
        2.03800630e-02,  1.79321393e-02,  3.93328704e-02,  2.29027569e-02,
       -7.47869164e-02, -6.23051897e-02, -5.15670627e-02, -1.77956582e-03,
       -2.63003260e-02,  8.11764523e-02,  3.68927675e-03, -1.29516264e-02,
       -3.68407667e-02, -4.24548431e-04, -1.80163011e-02, -2.99320891e-02,
       -3.34643610e-02, 

In [64]:
known_names = []
unknown_names = []
known_embeddings = []
unknown_embeddings = []

players = os.listdir('/Users/tayamayuto/Face_recognition_IF/data/ts_raw')

for player in tqdm(players):
    player_embeddings, player_names = [], []

    img_paths = glob(f'/Users/tayamayuto/Face_recognition_IF/data/ts_raw/{player}/*')
    for img_path in img_paths:
        img = cv2.imread(img_path)
        if img is None: continue

        bboxes, kpss = det_model.detect(img, max_num=0, metric='defualt')
        if len(bboxes) != 1: continue

        bbox = bboxes[0, :4]
        det_score = bboxes[0, 4]
        kps = kpss[0]
        face = Face(bbox=bbox, kps=kps, det_score=det_score)

        rec_model.get(img, face)
        player_embeddings.append(face.normed_embedding)
        player_names.append(player)
        if len(player_embeddings) == 10: break
    
    player_embeddings = np.stack(player_embeddings, axis=0)
    known_embeddings.append(player_embeddings[0:5])
    unknown_embeddings.append(player_embeddings[5:10])
    known_names += player_names[0:5]
    unknown_names += player_names[5:10]
    

known_embeddings = np.concatenate(known_embeddings, axis=0)
unknown_embeddings = np.concatenate(unknown_embeddings, axis=0)


 59%|█████▊    | 299/510 [04:37<03:32,  1.00s/it]libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
 84%|████████▍ | 430/510 [06:47<01:08,  1.16it/s]libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
100%|██████████| 510/510 [07:55<00:00,  1.07it/s]


In [66]:
print(face)

{'bbox': array([525.9521 ,  31.63786, 653.5623 , 204.3812 ], dtype=float32), 'kps': array([[552.03906 ,  99.21034 ],
       [605.9976  ,  94.14128 ],
       [573.89886 , 119.766815],
       [560.9429  , 160.75807 ],
       [603.84674 , 156.53514 ]], dtype=float32), 'det_score': 0.86615723, 'embedding': array([-4.74651545e-01,  4.59892482e-01, -1.29867911e-01, -7.61424780e-01,
       -8.06985676e-01,  5.46552241e-02,  9.45361137e-01,  6.18970335e-01,
       -2.62965083e-01, -1.24124813e+00, -1.13926601e+00, -5.65344095e-01,
       -1.33041537e+00,  7.12391675e-01, -5.68426847e-01,  2.12046757e-01,
        1.06422257e+00,  2.15615797e+00,  1.50813580e-01, -1.09571111e+00,
       -5.44137537e-01,  7.41333425e-01, -8.64532173e-01, -4.49727178e-01,
        1.94025326e+00,  1.74966240e+00, -1.23574853e+00, -1.34679687e+00,
       -2.69831806e-01, -8.65686297e-01,  1.78312659e-01,  9.69009697e-02,
        9.74745035e-01, -2.43629694e+00,  8.74355555e-01,  2.51986003e+00,
       -4.23281074e-0

In [59]:

def search_flatten(known_embeddings, known_names, unknown_embeddings, threshold=0.5):
    pred_names = []
    for emb  in unknown_embeddings:
        scores = np.dot(emb, known_embeddings.T)
        scores = np.clip(scores, 0., 1.)

        idx = np.argmax(scores)
        score = scores[idx]
        if score > threshold:
            pred_names.append(known_names[idx])
        else:
            pred_names.append(None)
    
    return pred_names

In [38]:
def get_averages(names, scores):
    d = defaultdict(list)
    for n, s in zip(names, scores):
        d[n].append(s)

    averages = {}
    for n, s in d.items():
        averages[n] = np.mean(s)
    
    return averages

In [39]:
def search_average(known_embeddings, known_names, unknown_embeddings, threshold=0.5):
    pred_names = []
    for emb in unknown_embeddings:
        scores = np.dot(emb, known_embeddings.T)
        scores = np.clip(scores, 0., 1.)

        averages = get_averages(known_names, scores)
        pred = sorted(averages, key=lambda x: averages[x], reverse=True)[0]
        score = averages[pred]

        if score > threshold:
            pred_names.append(pred)
        else:
            pred_names.append(None)
    
    return pred_names

In [40]:
def evaluate(true_names, pred_names):
    coverage = np.mean([n is not None for n in pred_names]) * 100.

    is_corrects = []
    for t, p in zip(true_names, pred_names):
        if p is None: continue
        is_corrects.append(t == p)
    
    if not is_corrects:
        is_corrects.append(False)

    accuracy = np.mean(is_corrects) * 100.
    return accuracy, coverage

In [62]:
pred_names = search_flatten(known_embeddings, known_names, unknown_embeddings, threshold=0.45)
acc_flatten, cov_flatten = evaluate(unknown_names, pred_names)

pred_names = search_average(known_embeddings, known_names, unknown_embeddings, threshold=0.45)
acc_average, cov_average = evaluate(unknown_names, pred_names)

results = pd.DataFrame([
    {'method': 'flatten', 'accuracy': acc_flatten, 'coverage': cov_flatten},
    {'method': 'average', 'accuracy': acc_average, 'coverage': cov_average},
])

print(results)

    method   accuracy   coverage
0  flatten  98.588443  96.838958
1  average  99.329823  89.235370


In [ ]:
def draw_on(img, faces, name):
    dimg = img.copy()
    for i in range(len(faces)):
        face = faces[i]
        box = face.bbox.astype(int)
        color = (0, 0, 255)
        cv2.rectangle(dimg, (box[0], box[1]), (box[2], box[3]), color, 2)
        if face.kps is not None:
            kps = face.kps.astype(int)
            #print(landmark.shape)
            for l in range(kps.shape[0]):
                color = (0, 0, 255)
                if l == 0 or l == 3:
                    color = (0, 255, 0)
                cv2.circle(dimg, (kps[l][0], kps[l][1]), 1, color, 2)
        cv2.putText(dimg, name, (box[0]-1, box[1]-4),cv2.FONT_HERSHEY_COMPLEX,1.5,(0,255,0),2)

    return dimg

app = FaceAnalysis()
app.prepare(ctx_id=1, det_size=(640, 640))

img_path = 'data/input/person1.JPG'

pre_img = cv2.imread(img_path)
pre_face = app.get(pre_img)
pre_embedding = [pre_face[0].embedding]

known_face_name = ["Unknown", "yuto"]

capture = cv2.VideoCapture(0)